# Partitioning Deep Dive — Hidden Partitioning & Partition Evolution

이 노트북에서는 Iceberg의 **파티셔닝** 전략을 실습합니다.

전통적 파티셔닝의 한계를 확인하고, Iceberg의 Hidden Partitioning과 Partition Evolution이 이를 어떻게 해결하는지 관찰합니다.

## 파티셔닝이란?

파티셔닝은 테이블의 데이터를 **논리적 그룹으로 나누어 저장**하는 것입니다.

쿼리 시 WHERE 조건에 해당하는 파티션만 읽으면 되므로, 불필요한 데이터를 건너뛸 수 있습니다 (**Partition Pruning**).

### Hive 스타일 파티셔닝의 문제점

| 문제 | 설명 |
|------|------|
| **파티션 폭증** | `PARTITIONED BY (order_date)` → 날짜가 60일이면 파티션 60개 |
| **파생 컬럼 필요** | 월별 파티션을 원하면 `order_month` 같은 컬럼을 직접 만들어야 함 |
| **사용자가 변환을 알아야 함** | 쿼리 시 파티션 컬럼으로 필터링해야 프루닝 동작 |
| **파티션 변경 불가** | 파티션 스키마를 바꾸면 전체 테이블 재작성 필요 |

## 환경 설정

In [1]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, snapshot_tree, diff_tree, count_files, total_size

In [2]:
spark = create_spark_session()

# 테이블 설정
IDENTITY_TABLE = "demo.lab.partition_identity"
IDENTITY_PATH = "/home/jovyan/data/warehouse/lab/partition_identity"

HIDDEN_TABLE = "demo.lab.partition_hidden"
HIDDEN_PATH = "/home/jovyan/data/warehouse/lab/partition_hidden"

EVOLUTION_TABLE = "demo.lab.partition_evolution"
EVOLUTION_PATH = "/home/jovyan/data/warehouse/lab/partition_evolution"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


---
## 실험 1: Identity Partition — 전통적 파티셔닝의 한계

`order_date` 컬럼으로 identity 파티셔닝하면, 날짜 값 하나마다 별도 파티션이 생성됩니다.

In [3]:
spark.sql(f"DROP TABLE IF EXISTS {IDENTITY_TABLE}")

spark.sql(f"""
CREATE TABLE {IDENTITY_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (order_date)
""")

print(f"Identity 파티션 테이블 생성 완료: {IDENTITY_TABLE}")

Identity 파티션 테이블 생성 완료: demo.lab.partition_identity


In [4]:
# 3개월(90일) 분량 데이터 삽입
orders = generate_orders(num_records=500, seed=42)
df = to_spark_df(spark, orders)
df.writeTo(IDENTITY_TABLE).append()

print(f"총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {IDENTITY_TABLE}').collect()[0][0]}")

총 레코드 수: 500


In [5]:
# Identity 파티션 구조 확인
print("Identity 파티션 파일 구조:")
print("=" * 60)
show_tree(IDENTITY_PATH, max_depth=2)

# 파티션 수 확인
partitions = spark.sql(f"SELECT * FROM {IDENTITY_TABLE}.partitions").collect()
print(f"\n파티션 수: {len(partitions)}")
print(f"Parquet 파일 수: {count_files(IDENTITY_PATH)}")

Identity 파티션 파일 구조:
├── data/
│   ├── order_date=2024-01-01/
│   ├── order_date=2024-01-02/
│   ├── order_date=2024-01-03/
│   ├── order_date=2024-01-04/
│   ├── order_date=2024-01-05/
│   ├── order_date=2024-01-06/
│   ├── order_date=2024-01-07/
│   ├── order_date=2024-01-08/
│   ├── order_date=2024-01-09/
│   ├── order_date=2024-01-10/
│   ├── order_date=2024-01-11/
│   ├── order_date=2024-01-12/
│   ├── order_date=2024-01-13/
│   ├── order_date=2024-01-14/
│   ├── order_date=2024-01-15/
│   ├── order_date=2024-01-16/
│   ├── order_date=2024-01-17/
│   ├── order_date=2024-01-18/
│   ├── order_date=2024-01-19/
│   ├── order_date=2024-01-20/
│   ├── order_date=2024-01-21/
│   ├── order_date=2024-01-22/
│   ├── order_date=2024-01-23/
│   ├── order_date=2024-01-24/
│   ├── order_date=2024-01-25/
│   ├── order_date=2024-01-27/
│   ├── order_date=2024-01-28/
│   ├── order_date=2024-01-29/
│   ├── order_date=2024-01-30/
│   ├── order_date=2024-01-31/
│   ├── order_date=2024-02-01/
│   ├── o

### 관찰 포인트 — Identity Partition

- 90일 범위의 데이터로 **최대 90개의 파티션 디렉토리**가 생성되었습니다
- 각 파티션에 소량의 데이터만 들어가 **Small File Problem** 발생
- 이는 Hive 스타일 파티셔닝에서 흔히 겪는 문제입니다

---
## 실험 2: Hidden Partitioning — 변환 함수로 파티션 축소

Iceberg는 **Hidden Partitioning**을 지원합니다. 파티션 컬럼을 별도로 만들지 않아도, `months()`, `days()`, `hours()`, `truncate()`, `bucket()` 등의 **변환 함수**를 사용하여 파티셔닝할 수 있습니다.

### 사용 가능한 변환 함수

| 변환 | 설명 | 예시 |
|------|------|------|
| `year(col)` | 연도 단위 | 2024 |
| `month(col)` | 연-월 단위 | 2024-01 |
| `day(col)` | 연-월-일 단위 | 2024-01-15 |
| `hour(col)` | 시간 단위 | 2024-01-15-08 |
| `truncate(col, N)` | 문자열/숫자를 N 단위로 자름 | truncate(id, 100) → 0, 100, 200... |
| `bucket(col, N)` | N개 버킷으로 해시 분배 | bucket(id, 16) → 0~15 |

In [6]:
spark.sql(f"DROP TABLE IF EXISTS {HIDDEN_TABLE}")

# months() 변환을 사용한 Hidden Partitioning
spark.sql(f"""
CREATE TABLE {HIDDEN_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

print(f"Hidden Partition 테이블 생성 완료: {HIDDEN_TABLE}")
print("파티션 변환: months(order_date) → 월 단위로 파티셔닝")

Hidden Partition 테이블 생성 완료: demo.lab.partition_hidden
파티션 변환: months(order_date) → 월 단위로 파티셔닝


In [7]:
# 동일한 데이터 삽입 (3개월 분량)
orders = generate_orders(num_records=500, seed=42)
df = to_spark_df(spark, orders)
df.writeTo(HIDDEN_TABLE).append()

print(f"총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {HIDDEN_TABLE}').collect()[0][0]}")

총 레코드 수: 500


In [8]:
# Hidden Partition 구조 확인
print("Hidden Partition 파일 구조:")
print("=" * 60)
show_tree(HIDDEN_PATH, max_depth=2)

# 파티션 수 비교
hidden_partitions = spark.sql(f"SELECT * FROM {HIDDEN_TABLE}.partitions").collect()
print(f"\nIdentity 파티션 수: {len(partitions)}")
print(f"Hidden 파티션 수:   {len(hidden_partitions)}")
print(f"Parquet 파일 수:    {count_files(HIDDEN_PATH)}")

Hidden Partition 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   ├── order_date_month=2024-02/
│   └── order_date_month=2024-03/
└── metadata/
    ├── 191ddb06-80e6-4e59-90a3-376d6eec267c-m0.avro  (7.3 KB)
    ├── snap-827375320685005734-1-191ddb06-80e6-4e59-90a3-376d6eec267c.avro  (4.2 KB)
    ├── v1.metadata.json  (1.5 KB)
    ├── v2.metadata.json  (2.6 KB)
    └── version-hint.text  (1 B)

Identity 파티션 수: 90
Hidden 파티션 수:   3
Parquet 파일 수:    3


In [9]:
# Partition Pruning 확인 — order_date로 필터링하면 자동으로 월 파티션 프루닝
print("EXPLAIN: order_date 필터 쿼리 (Hidden Partition 자동 프루닝)")
spark.sql(f"""
EXPLAIN EXTENDED
SELECT * FROM {HIDDEN_TABLE}
WHERE order_date >= '2024-02-01' AND order_date < '2024-03-01'
""").show(truncate=False)

EXPLAIN: order_date 필터 쿼리 (Hidden Partition 자동 프루닝)
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 관찰 포인트 — Hidden Partitioning

- `months(order_date)` 변환으로 **3개월 → 3개 파티션**만 생성되었습니다 (vs Identity의 ~90개)
- 사용자는 `order_date`로 필터링하면 Iceberg가 **자동으로 해당 월 파티션만 스캔**합니다
- 별도의 파티션 컬럼(`order_month`)을 만들 필요가 없습니다
- 쿼리 작성자가 파티셔닝 방식을 몰라도 프루닝이 동작합니다 — 이것이 **"Hidden"**의 의미입니다

---
## 실험 3: 변환 함수 비교 — year vs months vs days

같은 데이터에 서로 다른 변환을 적용했을 때 파티션 수가 어떻게 달라지는지 비교합니다.

In [10]:
# year, months, days 각각으로 파티셔닝한 테이블 생성
transforms = {
    "year":   "years(order_date)",
    "months": "months(order_date)",
    "days":   "days(order_date)",
}

for name, transform in transforms.items():
    table = f"demo.lab.partition_transform_{name}"
    spark.sql(f"DROP TABLE IF EXISTS {table}")
    spark.sql(f"""
    CREATE TABLE {table} (
        order_id BIGINT,
        customer_id BIGINT,
        product_name STRING,
        order_date DATE,
        amount DECIMAL(10,2),
        status STRING
    ) USING ICEBERG
    PARTITIONED BY ({transform})
    """)
    orders = generate_orders(num_records=500, seed=42)
    df = to_spark_df(spark, orders)
    df.writeTo(table).append()
    
    partition_count = len(spark.sql(f"SELECT * FROM {table}.partitions").collect())
    file_count = count_files(f"/home/jovyan/data/warehouse/lab/partition_transform_{name}")
    print(f"{name:8s} ({transform:25s}) → 파티션 {partition_count}개, 파일 {file_count}개")

year     (years(order_date)        ) → 파티션 1개, 파일 1개
months   (months(order_date)       ) → 파티션 3개, 파일 3개
days     (days(order_date)         ) → 파티션 90개, 파일 90개


### 관찰 포인트 — 변환 함수 비교

| 변환 | 예상 파티션 수 | 적합 사례 |
|------|---------------|----------|
| `years()` | ~1개 | 수년 단위 데이터, 매우 큰 테이블 |
| `months()` | ~3개 | 월 단위 분석이 빈번한 경우 (가장 일반적) |
| `days()` | ~90개 | 일 단위 정밀 프루닝이 필요한 경우 |

> **파티션 수의 황금률**: 파티션당 최소 수십~수백 MB의 데이터가 있도록 설계하세요.
> 파티션이 너무 많으면 Small File Problem, 너무 적으면 프루닝 효과 저하.

---
## 실험 4: Partition Evolution — 파티션 변경

Iceberg의 **Partition Evolution**은 기존 데이터를 재작성하지 않고 파티션 방식을 변경할 수 있게 해줍니다.

- 기존 데이터: 이전 파티션 스펙 유지
- 새 데이터: 새 파티션 스펙 적용
- 메타데이터에 여러 파티션 스펙이 공존

In [11]:
spark.sql(f"DROP TABLE IF EXISTS {EVOLUTION_TABLE}")

# 처음에는 year 파티셔닝으로 시작
spark.sql(f"""
CREATE TABLE {EVOLUTION_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (years(order_date))
""")

# 초기 데이터 삽입 (year 파티셔닝 적용)
orders = generate_orders(num_records=200, seed=10, start_date="2024-01-01", end_date="2024-03-31")
df = to_spark_df(spark, orders)
df.writeTo(EVOLUTION_TABLE).append()

print("[Phase 1] year 파티셔닝으로 200건 삽입")
print(f"파티션 수: {len(spark.sql(f'SELECT * FROM {EVOLUTION_TABLE}.partitions').collect())}")
show_tree(EVOLUTION_PATH, max_depth=2)

[Phase 1] year 파티셔닝으로 200건 삽입
파티션 수: 1
├── data/
│   └── order_date_year=2024/
└── metadata/
    ├── 648703bc-ed83-4c5a-b6d4-d12c478ac4c6-m0.avro  (7.1 KB)
    ├── snap-6663851353061963763-1-648703bc-ed83-4c5a-b6d4-d12c478ac4c6.avro  (4.2 KB)
    ├── v1.metadata.json  (1.5 KB)
    ├── v2.metadata.json  (2.6 KB)
    └── version-hint.text  (1 B)


In [12]:
# Partition Evolution: year → months로 변경
spark.sql(f"ALTER TABLE {EVOLUTION_TABLE} REPLACE PARTITION FIELD years(order_date) WITH months(order_date)")

print("파티션 스펙 변경 완료: years(order_date) → months(order_date)")
print("(기존 데이터는 재작성되지 않음 — 새 데이터만 새 스펙 적용)")

파티션 스펙 변경 완료: years(order_date) → months(order_date)
(기존 데이터는 재작성되지 않음 — 새 데이터만 새 스펙 적용)


In [13]:
# 새 데이터 삽입 (months 파티셔닝 적용)
orders_new = generate_orders(num_records=200, seed=20, start_date="2024-04-01", end_date="2024-06-30", id_offset=201)
df_new = to_spark_df(spark, orders_new)
df_new.writeTo(EVOLUTION_TABLE).append()

print("[Phase 2] months 파티셔닝으로 200건 추가 삽입")
print(f"총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {EVOLUTION_TABLE}').collect()[0][0]}")

[Phase 2] months 파티셔닝으로 200건 추가 삽입
총 레코드 수: 400


In [14]:
# Evolution 후 파일 구조 확인 — 두 스펙이 공존
print("Partition Evolution 후 파일 구조:")
print("=" * 60)
show_tree(EVOLUTION_PATH, max_depth=2)

print("\n파티션 정보:")
try:
    spark.sql(f"SELECT * FROM {EVOLUTION_TABLE}.partitions").show(truncate=False)
except Exception as e:
    print(f".partitions 조회 실패: {str(e).splitlines()[0]}")
    print("대체 출력: .files 기반 파티션 통계")
    spark.sql(f"""
    SELECT
        CAST(partition AS STRING) AS partition,
        COUNT(*) AS data_files,
        SUM(record_count) AS records,
        SUM(file_size_in_bytes) AS total_file_size_bytes
    FROM {EVOLUTION_TABLE}.files
    WHERE content = 0
    GROUP BY CAST(partition AS STRING)
    ORDER BY partition
    """).show(truncate=False)

Partition Evolution 후 파일 구조:
├── data/
│   ├── order_date_month=2024-04/
│   ├── order_date_month=2024-05/
│   ├── order_date_month=2024-06/
│   └── order_date_year=2024/
└── metadata/
    ├── 3548eeae-b0b2-421a-a5d6-be146bbdb0ea-m0.avro  (7.3 KB)
    ├── 648703bc-ed83-4c5a-b6d4-d12c478ac4c6-m0.avro  (7.1 KB)
    ├── snap-6057118507659139080-1-3548eeae-b0b2-421a-a5d6-be146bbdb0ea.avro  (4.3 KB)
    ├── snap-6663851353061963763-1-648703bc-ed83-4c5a-b6d4-d12c478ac4c6.avro  (4.2 KB)
    ├── v1.metadata.json  (1.5 KB)
    ├── v2.metadata.json  (2.6 KB)
    ├── v3.metadata.json  (2.9 KB)
    ├── v4.metadata.json  (3.9 KB)
    └── version-hint.text  (1 B)

파티션 정보:
.partitions 조회 실패: An error occurred while calling o274.showString.
대체 출력: .files 기반 파티션 통계
+-----------+----------+-------+---------------------+
|partition  |data_files|records|total_file_size_bytes|
+-----------+----------+-------+---------------------+
|{54, null} |1         |200    |4174                 |
|{null, 651}|1       

In [15]:
# 파티션 스펙 히스토리 확인 — 메타데이터에 여러 스펙 공존
import json
import glob

metadata_files = sorted(glob.glob(f"{EVOLUTION_PATH}/metadata/*.metadata.json"))
if metadata_files:
    latest = metadata_files[-1]
    with open(latest) as f:
        meta = json.load(f)
    
    print(f"메타데이터 파일: {latest.split('/')[-1]}")
    print(f"\n파티션 스펙 수: {len(meta.get('partition-specs', []))}")
    for spec in meta.get('partition-specs', []):
        print(f"  spec-id {spec['spec-id']}: {spec['fields']}")
    print(f"\n현재 기본 스펙 ID: {meta.get('default-spec-id')}")

메타데이터 파일: v4.metadata.json

파티션 스펙 수: 2
  spec-id 0: [{'name': 'order_date_year', 'transform': 'year', 'source-id': 4, 'field-id': 1000}]
  spec-id 1: [{'name': 'order_date_month', 'transform': 'month', 'source-id': 4, 'field-id': 1001}]

현재 기본 스펙 ID: 1


In [16]:
# spec-id별 파티션/파일 매핑 확인 — 어떤 파일이 어떤 파티션 스펙으로 쓰였는지
print("spec-id별 파티션 매핑 (.files 메타테이블):")
spark.sql(f"""
SELECT
    spec_id,
    CAST(partition AS STRING) AS partition_value,
    COUNT(*) AS data_files,
    SUM(record_count) AS records,
    MIN(regexp_replace(file_path, '^file:.*?/(data/)', '$1')) AS sample_file
FROM {EVOLUTION_TABLE}.files
WHERE content = 0
GROUP BY spec_id, CAST(partition AS STRING)
ORDER BY spec_id, partition_value
""").show(truncate=False)

print("\n해석:")
print("- spec_id = 0: year(order_date) 스펙으로 작성된 기존 파일")
print("- spec_id = 1: month(order_date) 스펙으로 작성된 신규 파일")
print("- default-spec-id는 앞으로 쓰일 기본 스펙이며, 기존 파일의 spec_id는 바뀌지 않음")


spec-id별 파티션 매핑 (.files 메타테이블):
+-------+---------------+----------+-------+--------------------------------------------------------------------------------------------------------------------------------+
|spec_id|partition_value|data_files|records|sample_file                                                                                                                     |
+-------+---------------+----------+-------+--------------------------------------------------------------------------------------------------------------------------------+
|0      |{54, null}     |1         |200    |data/warehouse/lab/partition_evolution/data/order_date_year=2024/00000-44-c04b5dfb-2703-408b-856d-da1e77f00ff7-00001.parquet    |
|1      |{null, 651}    |1         |80     |data/warehouse/lab/partition_evolution/data/order_date_month=2024-04/00000-51-f63df151-298e-4fd9-b062-256201552390-00001.parquet|
|1      |{null, 652}    |1         |61     |data/warehouse/lab/partition_evolution/data/order_date

### 관찰 포인트 — Partition Evolution

- 파티션 스펙을 `years` → `months`로 변경했지만, **기존 데이터는 재작성되지 않았습니다**
- 메타데이터에 **두 개의 파티션 스펙이 공존**합니다 (spec-id 0, 1)
- 어떤 파일/파티션이 어떤 스펙을 썼는지는 `table.files`의 `spec_id`로 확인합니다 (`GROUP BY spec_id, partition`)
- 기존 파일은 year 스펙, 새 파일은 months 스펙으로 기록됩니다
- `default-spec-id`는 **신규 write 기본값**이며, 기존 파일의 `spec_id`는 유지됩니다
- 일부 런타임 조합에서는 `table.partitions` 조회가 실패할 수 있어, 이 경우 `table.files` 집계로 파티션 상태를 확인합니다
- Iceberg 쿼리 엔진은 두 스펙을 모두 이해하고 올바르게 프루닝합니다
- Hive에서는 파티션 변경 시 전체 테이블을 재작성해야 했지만, Iceberg에서는 **메타데이터만 업데이트**하면 됩니다


---
## 실험 5: 메타데이터 테이블로 파티션 모니터링

Iceberg는 파티션 상태를 모니터링할 수 있는 **메타데이터 테이블**을 제공합니다.

In [17]:
# .partitions 메타데이터 테이블
print("=== .partitions — 파티션별 통계 ===")
spark.sql(f"SELECT * FROM {HIDDEN_TABLE}.partitions").show(truncate=False)

=== .partitions — 파티션별 통계 ===
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|last_updated_at        |last_updated_snapshot_id|
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+-----------------------+------------------------+
|{648}    |0      |159         |1         |3576                         |0                           |0                         |0                           |0                         |2026-02-16 00:11:33.251|827375320685005734      |
|{649}    |0      |161        

In [18]:
# .files 메타데이터 테이블 — 파일별 상세 정보
print("=== .files — 파일별 상세 정보 ===")
spark.sql(f"""
SELECT 
    partition,
    regexp_replace(file_path, '^file:.*?/(data/)', '$1') as file_path,
    file_format,
    record_count,
    file_size_in_bytes
FROM {HIDDEN_TABLE}.files
""").show(truncate=False)

=== .files — 파일별 상세 정보 ===
+---------+-----------------------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+
|partition|file_path                                                                                                                    |file_format|record_count|file_size_in_bytes|
+---------+-----------------------------------------------------------------------------------------------------------------------------+-----------+------------+------------------+
|{648}    |data/warehouse/lab/partition_hidden/data/order_date_month=2024-01/00000-14-480a11db-2e22-40fa-87a9-c326bb4d352b-00001.parquet|PARQUET    |159         |3576              |
|{649}    |data/warehouse/lab/partition_hidden/data/order_date_month=2024-02/00000-14-480a11db-2e22-40fa-87a9-c326bb4d352b-00002.parquet|PARQUET    |161         |3605              |
|{650}    |data/warehouse/lab/partition_hidden/data/order_date_

### 관찰 포인트 — 메타데이터 테이블

- `.partitions` 테이블로 **파티션별 레코드 수, 파일 수**를 확인할 수 있습니다
- `.files` 테이블로 **개별 파일의 크기, 레코드 수, 파티션 소속**을 확인할 수 있습니다
- 이를 통해 **불균형 파티션**(한 파티션에 데이터 편중)이나 **Small File Problem**을 사전에 감지할 수 있습니다

---
## Hive vs Iceberg 파티셔닝 비교

| 항목 | Hive 스타일 | Iceberg |
|------|-----------|--------|
| **파티션 컬럼** | 별도 컬럼 필요 (`order_month`) | 원본 컬럼 그대로 사용 |
| **변환** | 사용자가 직접 계산 | 변환 함수 자동 적용 (`months()`) |
| **쿼리** | 파티션 컬럼으로 필터링해야 프루닝 | 원본 컬럼 필터링 시 자동 프루닝 |
| **파티션 변경** | 전체 테이블 재작성 | 메타데이터만 업데이트 (Partition Evolution) |
| **파티션 스펙 공존** | 불가 | 가능 (여러 스펙 동시 존재) |
| **파티션 디스커버리** | 별도 `MSCK REPAIR TABLE` 필요 | 불필요 (메타데이터에 파일 목록 포함) |

### 핵심 요약

1. **Hidden Partitioning**: 파티션 컬럼 없이 변환 함수로 자동 파티셔닝 → 사용자 투명
2. **Partition Evolution**: 기존 데이터 재작성 없이 파티션 방식 변경 → 운영 유연성
3. **메타데이터 테이블**: `.partitions`, `.files`로 파티션 상태 모니터링 → 사전 진단

In [19]:
# 정리
for name in transforms:
    spark.sql(f"DROP TABLE IF EXISTS demo.lab.partition_transform_{name}")

spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
